# Parallel Iterative Methods (in Julia)

In this first example, we only have a single process. So, our print should happen once.

In [1]:
@everywhere println("Hello")

Hello


Lets add some processes, to make this parallel.

In [2]:
addprocs(4)
@everywhere println("Hej")

Hej
	From worker 2:	Hej
	From worker 5:	Hej
	From worker 4:	Hej
	From worker 3:	Hej


After we add 4 processes, we have 4 workers, and one master (the calling process). 

Here we've used the `@everywhere` macro to send a function to be evaluated by **every** process (including the master).

## What happens when we work with arrays?

In [8]:
@everywhere println(rand(2,2))

[0.093823 0.0328614; 0.941979 0.401809]
	From worker 4:	[0.10012 0.164494; 0.531035 0.492358]
	From worker 2:	[0.958769 0.19893; 0.947316 0.90248]
	From worker 3:	[0.716297 0.220145; 0.177109 0.83973]
	From worker 5:	[0.313709 0.50597; 0.988428 0.957271]


Predictibly, each process made their *own* random numbers.

This is **no good** for parallelizing iterative methods. 

Notice as well that the processes aren't executing in any particular order. 

This is a **major** feature that separates *parallel* programs from *sequential* programs.

## Shared Arrays

Let's initialize a _"Shared Array"_ using the workers, to be shared amongst the workers.

In [9]:
A = SharedArray(Int64, 
                (4,5), 
                init = S::SharedArray -> S[indexpids(S):length(procs(S)):length(S)] = myid())

4×5 SharedArray{Int64,2}:
 2  2  2  2  2
 3  3  3  3  3
 4  4  4  4  4
 5  5  5  5  5

Above, the work of initializing was split evenly among the 4 workers. 

In a **20** element matrix, each processor got **5** to initialize (value in the matrix = ID of the assigned worker). 

The `indexpids` function was used to set the start index 
(values 1, 2, 3, or 4 depending on the processor), 
and the rest of the range sets the _stride_.

In [5]:
for p in procs(A)
    @async println(fetch(remotecall_wait(indexpids, p, A)))
end

1
4
2
3


# Parallel Jacobi

_The Method of Simultaneous Displacements_